In [129]:
from functools import reduce

In [30]:
class node:

    def __init__(self):
        self.up = self
        self.down = self
        self.left = self
        self.right = self
        self.COLUMN = None
        self.ROW = None

    def hideInCol(self):
        self.up.down = self.down
        self.down.up = self.up
        return self

    def showIncol(self):
        self.up.down = self
        self.down.up = self
        return self

    def hideInCol(self):
        self.left.right = self.right
        self.right.left = self.left
        return self

    def showInCol(self):
        self.left.right = self
        self.right.left = self
        return self

    def hideWholeRow(self, start = None):
        if start == None: # if we just started the loop, don't hide me from my column, but go to my neighbor
            self.right.hideWholeRow(start = self)
            return
        if start != self: # we are not first and not last => hide me and continue
            self.hideInCol()
            self.right.hideWholeRow(start = start)
            return
        return # last possibility start == self => we have looped and can finish
    
    def getNeighbor(self, direction, by=0):
        neighbor = {
            'left'  : self.left,
            'right' : self.right,
            'up'    : self.up,
            'down'  : self.down
        }[direction]
        if by > 0:
            return neighbor.getNeighbor(direction, by = by-1)
        return neighbor




In [31]:
class columnHeader(node):

    def __init__(self, label):
        super().__init__()
        self.column_label = label
        self.row_count = 0
        self.COL = self

    def hideMyRows(self):
        cell = self.down
        while cell != self:
            cell.hideWholeRow()
            cell = cell.down
        return self

    def showMyRows(self):
        cell = self.down
        while cell != self:
            cell.showWholeRow()
            cell = cell.down
        return self

    def appendRow(self, row):
        row.down = self
        row.up = self.up
        self.up.down = row
        self.up = row
        return self

    def addToRoot(self, root):
        root.appendColumn(self)
        return self

In [32]:
class rowHeader(node):
    
    def __init__(self, label):
        super().__init__()
        self.row_label = label
        self.ROW = self

    def appendColumn(self, col):
        col.right = self
        col.left = self.left
        self.left.right = col
        self.left = col
        return self

    def addToRoot(self, root):
        root.appendRow(self)
        return self


In [33]:
class rootNode(columnHeader, rowHeader):

    def __init__(self):
        node.__init__(self)
        self.ROW = self
        self.COL = self

    def addToRoot(self, root):
        pass

In [51]:
matrix_root = rootNode()

# generate column header labels 
constraint_type = {
    'cell' : {
        'label_template' : 'R{i}C{j}',
        'dim_1' : 'R',
        'dim_2' : 'C',
    },
    'row' : {
        'label_template' : 'R{i}#{j}',
        'dim_1' : 'R',
        'dim_2' : '#',
    },
    'col' : {
        'label_template' : 'C{i}#{j}',
        'dim_1' : 'C',
        'dim_2' : '#',
    },
    'box' : {
        'label_template' : 'B{i}#{j}',
        'dim_1' : 'B',
        'dim_2' : '#',
    },
}

column_headers = { 
    c : {
        i : {
            j : columnHeader(constraint_type[c]['label_template'].format(i = i, j = j)).addToRoot(matrix_root)
            for j in range(1,10)
        }
        for i in range(1, 10)
    }
    for c in constraint_type
}

In [84]:
# generate row labels

row_labels = {
    i : {
        j : {
            k : {
                'R' : i,
                'C' : j,
                'B' : (j-1) // 3 + 3 * ((i-1) // 3) + 1,
                '#' : k,
                'label' : f'R{i}C{j}#{k}'
            }
            for k in range(1,10)
        }
        for j in range(1,10)
    }
    for i in range(1,10)
}

In [88]:
# construct the sparse matrix

for row_grp in row_labels.values():
    for row_set in row_grp.values():
        for row in row_set.values():

            row_header = rowHeader(row)
            matrix_root.appendRow(row_header)

            for c_name, c_specs in constraint_type.items():

                cell = node()
                row_header.appendColumn(cell)
                cell.ROW = row_header
                column_header = column_headers[c_name][row[c_specs['dim_1']]][row[c_specs['dim_2']]]
                cell.COL = column_header
                column_header.appendRow(cell)

            row['obj'] = row_header



In [150]:
# file I/O utilities

def read_sudoku(filename):
    with open(filename, 'r') as f:
        lines = f.readlines(121)
        print(len(lines))
    tab = [
        {
            'R' : l_id + 1,
            'C' : ch_id + 1,
            '#' : ch
        }
        for l_id, line in enumerate(lines)
        for ch_id, ch in enumerate(line)
        if (ch != '-' and ch != '\n')
    ]
    return tab

def write_sudoku(solution, filename):
    tab = [['-' for i in range(1,10)] for j in range(1,10)]
    for item in solution:
        tab[item['R']-1][item['C']-1] = str(item['#'])
    
    res = '\n'.join([
        ''.join(line) for line in tab
    ])
    return res


In [151]:
x = read_sudoku('input_1.sudoku')

3


In [152]:
write_sudoku(x, 'x')

'--1--2---\n-3--5----\n-4-7-8---\n---------\n---------\n---------\n---------\n---------\n---------'